# **Tools: common functions for training and testing**

In [72]:
from PIL import Image
import numpy as np
import os
import cv2
import keras


# Making images into array
def imgToArray(rootdir):
    data = []
    labels = []
    for subdir, dirs, files in os.walk(rootdir):
        for dir in dirs:
            currDir = os.path.join(subdir, dir)
            for file in os.listdir(currDir):
                if ".jpg" in file:
                    imag = cv2.imread(os.path.join(currDir, file))
                    img_from_ar = Image.fromarray(imag, 'RGB')
                    resized_image = img_from_ar.resize((50, 50))
                    data.append(np.array(resized_image))
                    labels.append(int(dir)-1)
    return data, labels


# Convert data and labels to numpy arrays
def convertToNumpy(data, labels):
    cows=np.array(data)
    labels=np.array(labels)
    return cows,labels


# Create data - X and Y
def createData(cows, labels):
    # Ensure that the input features are scaled between 0.0 and 1.0
    x_data = cows.astype('float32')/255
    # One hot encoding
    num_classes = len(np.unique(labels))
    y_data=keras.utils.to_categorical(labels,num_classes)
    return x_data, y_data


# **Training:**

In [74]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

# -------------------------- Prepare Train Set --------------------------
# Making images into array
def train_imgToArray():
    rootdirTrain = '/content/datasets/train'
    dataTrain, labelsTrain = imgToArray(rootdirTrain)
    return dataTrain, labelsTrain


# Convert dataTrain and labelsTrain to numpy arrays
def train_convertToNumpy(dataTrain, labelsTrain):
    cowsTrain, labelsTrain = convertToNumpy(dataTrain, labelsTrain)
    return cowsTrain,labelsTrain


# Create data - X and Y
def train_createData(cowsTrain, labelsTrain):
    x_train, y_train = createData(cowsTrain, labelsTrain)
    return x_train, y_train


# -------------------------- Making Keras model --------------------------
def kerasModel():
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(46,activation="softmax"))
    model.summary()

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model


# -------------------------- Train the model --------------------------
def training(model, x_train, y_train):
    model.fit(x_train,y_train,batch_size=50,epochs=5,verbose=1)
    return model

# **Testing:**

In [75]:
# -------------------------- Prepare Test Set --------------------------
# Making images into array
def test_imgToArray():
    rootdirTest = '/content/datasets/test'
    dataTest, labelsTest = imgToArray(rootdirTest)
    return dataTest,labelsTest


# Convert dataTest and labelsTest to numpy arrays
def test_convertToNumpy(dataTest, labelsTest):
    cowsTest, labelsTest = convertToNumpy(dataTest, labelsTest)
    return cowsTest, labelsTest

def test_createData(cowsTest, labelsTest):
    x_test, y_test = createData(cowsTest, labelsTest)
    return x_test, y_test


# -------------------------- Test the model --------------------------
def testing(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=1)
    print('\n', 'Test accuracy:', score[1])


# **Main:**

In [77]:
dataTrain, labelsTrain = train_imgToArray()
cowsTrain, labelsTrain = train_convertToNumpy(dataTrain, labelsTrain)
x_train, y_train = train_createData(cowsTrain, labelsTrain)
model = kerasModel()
model = training(model, x_train, y_train)

dataTest, labelsTest = test_imgToArray()
cowsTest, labelsTest = test_convertToNumpy(dataTest, labelsTest)
x_test, y_test = test_createData(cowsTest, labelsTest)
testing(model, x_test, y_test)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 6, 6, 64)         